# Multiple WHERE clauses

Semi joins in the form of nested subqueries are within the WHERE clause of the main query. In this exercise, you'll familiarize yourself with semi join syntax by thinking through and re-ordering the lines of code provided. Note that subqueries are queries in their own right, so they can have a WHERE clause of their own! This is why you see two WHERE statements here.

Your task is to construct a semi join that pulls all records from economies2019 where gross_savings in the economies2015 table were below the 2015 global average. The global average gross_savings in 2015 was 22.5, and is already pre-calculated in the lines of code provided.

<img src="images/04.02.jpg" style="width:800px;height:400px;">

# Semi join

You are interested in identifying languages spoken in the Middle East. The languages table contains information about languages and countries, but it does not tell you what region the countries belong to. You can build up a semi join by filtering the countries table by a particular region, and then using this to further filter the languages table.

You'll build up your semi join as you did in the video exercise, block by block, starting with a selection of countries from the countries table, and then leveraging a WHERE clause to filter the languages table by this selection.

```
SELECT DISTINCT name
FROM languages
-- Add syntax to use bracketed subquery below as a filter
WHERE code IN
    (SELECT code
    FROM countries
    WHERE region = 'Middle East')
ORDER BY name;
```

# Diagnosing problems using anti join

The anti join is a related and powerful joining tool. It can be particularly useful for identifying whether an incorrect number of records appears in a join.

Say we are interested in identifying currencies of Oceanian countries. We have the following INNER JOIN which returns 15 records, and want to check that all Oceanian countries in our countries table are included in this result. If not, we want to return the names of any excluded countries.
```
SELECT c1.code, name, basic_unit AS currency
FROM countries AS c1
INNER JOIN currencies AS c2
ON c1.code = c2.code
WHERE c1.continent = 'Oceania';
```
An anti join will give us the names of these countries, if any! 

```
SELECT code, name
FROM countries
WHERE continent = 'Oceania'
-- Filter for countries not included in the bracketed subquery
  AND code NOT IN
    (SELECT code
    FROM currencies);
```

# Subquery inside WHERE

 subqueries inside WHERE can either be from the same table or a different table. In this exercise, you will nest a subquery from the populations table inside another query, also from the populations table. Your goal is to figure out which countries had high average life expectancies in 2015.

You can use SQL to do calculations for you. Suppose you only want records from 2015 with life_expectancy above 1.15 * avg_life_expectancy. You could use the following SQL query:
```
SELECT *
FROM populations
WHERE life_expectancy > 1.15 * avg_life_expectancy
  AND year = 2015;
```  

```
SELECT *
FROM populations
-- Filter for only those populations where life expectancy is 1.15 times higher than average
WHERE life_expectancy > 1.15 *
  (SELECT AVG(life_expectancy)
   FROM populations
   WHERE year = 2015) 
	 AND year = 2015;
```

# WHERE do people live?

you will strengthen your knowledge of subquerying using WHERE. Follow the instructions below to get the urban area population for capital cities only. Explore the tables displayed in the console to help identify columns of interest as you build your query.

```
-- Select relevant fields from cities table
SELECT  name, country_code, urbanarea_pop
-- Filter using a subquery on the countries table
FROM cities
WHERE name IN ( SELECT capital FROM countries)
ORDER BY urbanarea_pop DESC;
```

# Subquery inside SELECT

There are often multiple ways to produce the same result in SQL. You saw that subqueries can provide an alternative to joins to obtain the same result.

you'll go further in exploring how some queries can be written using either a join or a subquery.

In Step 1, you'll begin with a LEFT JOIN combined with a GROUP BY to obtain summarized information from two tables in order to select the nine countries with the most cities appearing in the cities table. In Step 2, you'll write a query that returns the same result as the join, but leveraging a nested query instead.

```
-- Find top nine countries with the most cities
SELECT countries.name AS country, COUNT(cities.name) AS cities_num
FROM countries
LEFT JOIN cities
ON countries.code = cities.country_code
-- Order by count of cities as cities_num
GROUP BY country
ORDER BY cities_num DESC
LIMIT 9;
```

```
SELECT countries.name AS country,
-- Subquery that provides the count of cities 
  (SELECT COUNT(*) 
  FROM cities 
  WHERE countries.code = cities.country_code) AS cities_num
FROM countries

ORDER BY cities_num DESC, country
LIMIT 9;
```

# Subquery inside FROM

Subqueries inside FROM can help us select columns from multiple tables in a single query.

Say you are interested in determining the number of languages spoken for each country. You want to present this information alongside each country's local_name, which is a field only present in the countries table and not in the languages table. You'll use a subquery inside FROM to bring information from these two tables together!

```
-- Select code, and language count as lang_num
SELECT languages.code, COUNT(languages.name) AS lang_num
FROM languages
GROUP BY languages.code
```

```
-- Select local_name and lang_num from appropriate tables
SELECT local_name, lang_num
FROM countries,
  (SELECT code, COUNT(*) AS lang_num
  FROM languages
  GROUP BY code) AS sub
-- Where codes match
WHERE countries.code = sub.code
ORDER BY lang_num DESC;
```

# Subquery challenge

you're interested in analyzing inflation and unemployment rate for certain countries in 2015. You are not interested in countries with "Republic" or "Monarchy" as their form of government, but are interested in all other forms of government, such as emirate federations, socialist states, and commonwealths.

You will use the field gov_form to filter for these two conditions, which represents a country's form of government. You can review the different entries for gov_form in the countries table.

```
-- Select relevant fields
SELECT code, inflation_rate, unemployment_rate
FROM economies
WHERE year = 2015 
  AND code NOT IN
-- Subquery returning country codes filtered on gov_form
	(SELECT code 
  FROM countries 
  WHERE gov_form LIKE '%epublic%' OR gov_form LIKE '%onarch%' )
ORDER BY inflation_rate;
```

# Final challenge

Determine the top 10 capital cities in Europe and the Americas by city_perc, a metric you'll calculate. city_perc is a percentage that calculates the "proper" population in a city as a percentage of the total population in the wider metro area, as follows:

city_proper_pop / metroarea_pop * 100

```
-- Select fields from cities
SELECT cities.name, cities.country_code, cities.city_proper_pop, cities.metroarea_pop, (cities.city_proper_pop /cities.metroarea_pop * 100) AS city_perc
FROM cities
-- Use subquery to filter city name
WHERE cities.name IN (SELECT capital 
FROM countries
WHERE (countries.continent LIKE 'Europe' OR countries.continent LIKE '%America')
-- Add filter condition such that metroarea_pop does not have null values
AND cities.metroarea_pop IS NOT NULL)
-- Sort and limit the result
ORDER BY city_perc DESC
LIMIT 10
```